In [1]:
import pandas as pd
import requests
from io import StringIO 
from lxml import etree
import csv

In [2]:
class Downloader:
    
    def __init__(self, url):
        self.url = url
        self.html = ''
    
    def download_page(self):
        req = requests.get(self.url)
        self.html = req.text
        return 

In [4]:
r = Downloader('https://boardgamegeek.com/browse/boardgame/page1')

In [5]:
r.download_page()

In [6]:
r.url

'https://boardgamegeek.com/browse/boardgame/page1'

In [7]:
r.html

'\n<!DOCTYPE html>\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\n<head>\n\t<meta charset=\'utf-8\'>\n\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t\t<script>\n\t\t\twindow.addEventListener( \'DOMContentLoaded\',  function() {\n\t\t\t\tif (window.innerWidth < 960) {\n\t\t\t\t\tvar mvp = document.getElementById(\'vp\');\n\t\t\t\t\tmvp.setAttribute(\'content\',\'width=960\');\n\t\t\t\t}\n\t\t\t});\n\t\t</script>\n\t\t<meta content=\'yes\' name=\'apple-mobile-web-app-capable\'>\n\t<meta content=\'IE=edge,chrome=1\' http-equiv=\'X-UA-Compatible\'>\n\n\t\t\t<title>Browse Board Games | BoardGameGeek</title>\n\t\n\t\n<link rel="apple-touch-icon" \thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\n<link rel="shortcut icon" \t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="icon" \t\t\t\t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="search" \t\t\t\thref="/gam

In [3]:
class Parser:
    
    xpath_celdas_nombre = "//table[@id='collectionitems']/tr/td[3]/div[2][contains(@id,'results_objectname')]"
    xpath_geek = "//table[@id='collectionitems']/tr/td[4]/text()"
    xpath_average = "//table[@id='collectionitems']/tr/td[5]/text()"
    xpath_voters = "//table[@id='collectionitems']/tr/td[6]/text()"
    
    def __init__(self, html):
        
        self.html = html
        self.tabla = []
        
        self.htmlparser = etree.HTMLParser()
        self.html_tree = etree.parse(StringIO(html), self.htmlparser)
                
    def clean_entero(self, string_entero):
        if "N/A" in string_entero:
            return
        else:
            return int(string_entero.replace('(','').replace(')',''))
    
    def clean_flotante(self, string_flotante):
        if "N/A" in string_flotante:
            return
        else: 
            return float(string_flotante.replace('\n','').replace('\t',''))
    
    def download_tabla(self):
        
        celdas_nombre = self.html_tree.xpath(self.xpath_celdas_nombre)

        name = []
        year_string = []
        
        for c in celdas_nombre:
            n = c.xpath("a/text()")
            name += n
            year = c.xpath("span/text()")
            if year:
                year_string += year
            else:
                year_string.append("N/A")

        geek_string = self.html_tree.xpath(self.xpath_geek)
        average_string = self.html_tree.xpath(self.xpath_average)
        voters_string = self.html_tree.xpath(self.xpath_voters)

        year = list(map(self.clean_entero,year_string))
        voters = list(map(self.clean_entero,voters_string))

        geek = list(map(self.clean_flotante,geek_string))
        average= list(map(self.clean_flotante,average_string))

        self.tabla = list(zip(name,year,geek,average,voters))
        return self.tabla

In [6]:
p = Parser(r.html)

In [12]:
p.download_tabla()

[('Gloomhaven', 2017, 8.582, 8.85, 31932),
 ('Pandemic Legacy: Season 1', 2015, 8.47, 8.62, 35062),
 ('Terraforming Mars', 2016, 8.268, 8.42, 49308),
 ('Through the Ages: A New Story of Civilization', 2015, 8.234, 8.49, 18498),
 ('Brass: Birmingham', 2018, 8.218, 8.63, 10468),
 ('Twilight Imperium (Fourth Edition)', 2017, 8.175, 8.69, 9629),
 ('Twilight Struggle', 2005, 8.16, 8.31, 36864),
 ('Star Wars: Rebellion', 2016, 8.157, 8.43, 19225),
 ('Gaia Project', 2017, 8.141, 8.5, 12262),
 ('Great Western Trail', 2016, 8.097, 8.28, 23451),
 ('Scythe', 2016, 8.097, 8.26, 47215),
 ('War of the Ring (Second Edition)', 2012, 8.066, 8.45, 11327),
 ('Spirit Island', 2017, 8.032, 8.32, 16691),
 ('Terra Mystica', 2012, 8.02, 8.17, 35187),
 ('The Castles of Burgundy', 2011, 8.011, 8.13, 40525),
 ('7 Wonders Duel', 2015, 7.979, 8.11, 47723),
 ('The 7th Continent', 2017, 7.967, 8.35, 13050),
 ('Concordia', 2013, 7.946, 8.11, 23208),
 ('Brass: Lancashire', 2007, 7.938, 8.15, 16806),
 ('Viticulture Ess

In [18]:
f = Downloader('https://boardgamegeek.com/browse/boardgame/page2')

In [14]:
def guardar_csv(tabla):
    with open('tabla_20200131.csv', 'w', encoding='utf-8') as myfile:
     writer = csv.writer(myfile)
     writer.writerow(['name', 'year', 'geek_rating','average_rating','voters'])
     writer.writerows(tabla)
    return

In [16]:
tabla_completa = []

for i in range(1, 1138):
    print(f'bajando pagina {i}')
    r = Downloader(f'https://boardgamegeek.com/browse/boardgame/page/{i}')
    r.download_page()
    p = Parser(r.html)
    p.download_tabla()
    tabla_completa += p.tabla

guardar_csv(tabla_completa)

bajando pagina 1
bajando pagina 2
bajando pagina 3
bajando pagina 4
bajando pagina 5
bajando pagina 6
bajando pagina 7
bajando pagina 8
bajando pagina 9
bajando pagina 10
bajando pagina 11
bajando pagina 12
bajando pagina 13
bajando pagina 14
bajando pagina 15
bajando pagina 16
bajando pagina 17
bajando pagina 18
bajando pagina 19
bajando pagina 20
bajando pagina 21
bajando pagina 22
bajando pagina 23
bajando pagina 24
bajando pagina 25
bajando pagina 26
bajando pagina 27
bajando pagina 28
bajando pagina 29
bajando pagina 30
bajando pagina 31
bajando pagina 32
bajando pagina 33
bajando pagina 34
bajando pagina 35
bajando pagina 36
bajando pagina 37
bajando pagina 38
bajando pagina 39
bajando pagina 40
bajando pagina 41
bajando pagina 42
bajando pagina 43
bajando pagina 44
bajando pagina 45
bajando pagina 46
bajando pagina 47
bajando pagina 48
bajando pagina 49
bajando pagina 50
bajando pagina 51
bajando pagina 52
bajando pagina 53
bajando pagina 54
bajando pagina 55
bajando pagina 56
b

bajando pagina 438
bajando pagina 439
bajando pagina 440
bajando pagina 441
bajando pagina 442
bajando pagina 443
bajando pagina 444
bajando pagina 445
bajando pagina 446
bajando pagina 447
bajando pagina 448
bajando pagina 449
bajando pagina 450
bajando pagina 451
bajando pagina 452
bajando pagina 453
bajando pagina 454
bajando pagina 455
bajando pagina 456
bajando pagina 457
bajando pagina 458
bajando pagina 459
bajando pagina 460
bajando pagina 461
bajando pagina 462
bajando pagina 463
bajando pagina 464
bajando pagina 465
bajando pagina 466
bajando pagina 467
bajando pagina 468
bajando pagina 469
bajando pagina 470
bajando pagina 471
bajando pagina 472
bajando pagina 473
bajando pagina 474
bajando pagina 475
bajando pagina 476
bajando pagina 477
bajando pagina 478
bajando pagina 479
bajando pagina 480
bajando pagina 481
bajando pagina 482
bajando pagina 483
bajando pagina 484
bajando pagina 485
bajando pagina 486
bajando pagina 487
bajando pagina 488
bajando pagina 489
bajando pagi

bajando pagina 870
bajando pagina 871
bajando pagina 872
bajando pagina 873
bajando pagina 874
bajando pagina 875
bajando pagina 876
bajando pagina 877
bajando pagina 878
bajando pagina 879
bajando pagina 880
bajando pagina 881
bajando pagina 882
bajando pagina 883
bajando pagina 884
bajando pagina 885
bajando pagina 886
bajando pagina 887
bajando pagina 888
bajando pagina 889
bajando pagina 890
bajando pagina 891
bajando pagina 892
bajando pagina 893
bajando pagina 894
bajando pagina 895
bajando pagina 896
bajando pagina 897
bajando pagina 898
bajando pagina 899
bajando pagina 900
bajando pagina 901
bajando pagina 902
bajando pagina 903
bajando pagina 904
bajando pagina 905
bajando pagina 906
bajando pagina 907
bajando pagina 908
bajando pagina 909
bajando pagina 910
bajando pagina 911
bajando pagina 912
bajando pagina 913
bajando pagina 914
bajando pagina 915
bajando pagina 916
bajando pagina 917
bajando pagina 918
bajando pagina 919
bajando pagina 920
bajando pagina 921
bajando pagi

In [17]:
juegos = pd.read_csv('tabla_20200131.csv', encoding = "ISO-8859-1")
juegos.head()

,name,year,geek_rating,average_rating,voters
0,Gloomhaven,2017.0,8.582,8.85,31932.0
1,Pandemic Legacy: Season 1,2015.0,8.470,8.62,35062.0
2,Terraforming Mars,2016.0,8.268,8.42,49308.0
3,Through the Ages: A New Story of Civilization,2015.0,8.234,8.49,18498.0
4,Brass: Birmingham,2018.0,8.218,8.63,10468.0


In [18]:
type(r)

__main__.Downloader

In [19]:
isinstance(1, int)

True

In [20]:
isinstance("abv", float)

False

In [21]:
isinstance(r, Downloader)

True

In [8]:
p.xpath_celdas_nombre

"//table[@id='collectionitems']/tr/td[3]/div[2][contains(@id,'results_objectname')]"

In [10]:
p.htmlparser

In [11]:
p.html_tree